## Rasa nlu

## Clone git repo

In [1]:
BRANCH_NAME='dev'
! git clone -b $BRANCH_NAME https://github.com/CiscoAI/cisco-kubeflow-starter-pack.git

Cloning into 'cisco-kubeflow-starter-pack'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 3346 (delta 11), reused 50 (delta 6), pack-reused 3276
Receiving objects: 100% (3346/3346), 12.08 MiB | 41.36 MiB/s, done.
Resolving deltas: 100% (1269/1269), done.


## Install libraries

In [2]:
!pip install kfp --user

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


## Restart Notebook Kernel

In [ ]:
from IPython.display import display_html
display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)

## Import libraries

In [1]:
import kfp
import os
from kubernetes import client
from datetime import datetime

## Component files Declarations

In [2]:
path='cisco-kubeflow-starter-pack/apps/healthcare/covid-forecasting/onprem/pipelines/'
component_root_core_train= path+'components/rasa-core-train/'
component_root_nlu_train = path+'components/rasa-nlu-train/'
component_root_evaluate = path+'components/rasa-evaluate/'

## Load components from respective .YAML config files

In [3]:
rasa_core_op = kfp.components.load_component_from_file(os.path.join(component_root_core_train, 'component.yaml'))
rasa_nlu_op = kfp.components.load_component_from_file(os.path.join(component_root_nlu_train, 'component.yaml'))
rasa_evaluate_op = kfp.components.load_component_from_file(os.path.join(component_root_evaluate, 'component.yaml'))

## Define Volume and Volume Mounts

In [4]:
nfs_pvc = client.V1PersistentVolumeClaimVolumeSource(claim_name='nfs')
nfs_volume = client.V1Volume(name='nfs', persistent_volume_claim=nfs_pvc)
nfs_volume_mount = client.V1VolumeMount(mount_path='/mnt/', name='nfs')

## Define pipeline function

In [5]:
def rasa_pipeline(
    base="/ml",
    models_dir="models",
    domain_yml="domain.yml",
    stories_md="data/stories.md",
    dialogue_model_name="dialogue",
    nlu_config_yml="config.yml",
    data_md="data/nlu.md",
    nlu_model_name="nlu",
    project_name="current",
    eval_stories_md = "tests/conversation_tests.md",
    results_dir="results",
    output_dir="/mnt"
    ):
    
    models_path = str(output_dir) + "/" + str(models_dir)
    domain = str(base) + "/" + str(domain_yml)
    stories = str(base) + "/" + str(stories_md)
    dialogue_model_dir = str(models_path) + "/" + str(dialogue_model_name)
    
    nlu_config = str(base) + "/" + str(nlu_config_yml)
    data = str(base) + "/" + str(data_md)
    
    eval_stories_dir = str(base) + "/" + str(eval_stories_md)
    nlu_model_dir = str(models_path) + "/" + str(project_name) + "/" + str(nlu_model_name)
    results = str(models_path) + "/" + str(results_dir)
    
    core_train = rasa_core_op(domain, stories, dialogue_model_dir)
    core_train.add_volume(nfs_volume)
    core_train.add_volume_mount(nfs_volume_mount)
    
    nlu_train = rasa_nlu_op(nlu_config, data, models_path, nlu_model_name, project_name)
    nlu_train.add_volume(nfs_volume)
    nlu_train.add_volume_mount(nfs_volume_mount)
    nlu_train.after(core_train)
    
    eval = rasa_evaluate_op(dialogue_model_dir, nlu_model_dir, eval_stories_dir, results)
    eval.add_volume(nfs_volume)
    eval.add_volume_mount(nfs_volume_mount)
    eval.after(nlu_train)

In [6]:
try:
    import kfp.compiler as compiler
    compiler.Compiler().compile(rasa_pipeline, 'rasa_pipeline.tar.gz')
except RuntimeError as err:
    logging.debug(err)
    logging.info("Argo workflow failed validation check but it can still be used to run experiments.")

## Create kubeflow pipeline experiment

In [7]:
client = kfp.Client()
rasa_experiment = client.create_experiment(name='RASA')

## Create kubeflow pipeline run

In [8]:
timestamp = datetime.now().strftime("%d_%m_%y_%H_%M_%S")
run = client.run_pipeline(rasa_experiment.id, 'rasa-pipeline-'+timestamp, pipeline_package_path='rasa_pipeline.tar.gz')